<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Understanding Ocean Deoxygenation</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [1]:
# ---------- Librairies ----------
import matplotlib.pyplot as plt
from pathlib import Path
import xarray as xr

# ---------- Jupyter ----------
%matplotlib inline
plt.rcParams.update({"font.size": 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Scripts</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
%run scripts/run_training.py -b async -c scripts/configs/debug.yml

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">